In [21]:
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker import HierarchicalChunker
import torch
from rag_configuration import CFG
from pathlib import Path
import re
from utils import ExtractedContent
from transformers import AutoModel, AutoTokenizer
import numpy as np
from datasets import Dataset
from utils.text_preprocessing import contains_valid_word_like_sequence
torch.cuda.empty_cache()
torch.set_default_device('cuda')

In [2]:
torch.cuda.is_available()


True

In [3]:
embedding_model = AutoModel.from_pretrained(CFG.embedding_model_name, trust_remote_code=True,
                                            torch_dtype=torch.bfloat16, use_flash_attn=False).to('cuda')
embedding_tokenizer = AutoTokenizer.from_pretrained(CFG.embedding_model_name)
embedding_tokenizer.model_max_length = 8192

In [4]:
document_paths: list[str] = [str(path) for path in Path(CFG.data_dir).glob('**/*.pdf')]

In [5]:
def get_title(pdf_path:str) -> str:
        return re.search(CFG.filename_pattern, pdf_path).group(1).replace('-', ' ').replace('_', ' ')

In [6]:
def get_section_titles(current_page:int,page_mapping:dict[str, int]):
    smaller_page_mapping = {k:v for k,v in page_mapping.items() if v <= current_page}
    if smaller_page_mapping.values():
        matching_value = max(list(smaller_page_mapping.values()))
        return ', '.join([key for key in page_mapping if page_mapping[key]==matching_value])
    return ''

In [7]:
converter = DocumentConverter()

In [8]:
torch.set_default_device('cpu')
final_data_list = []
for document_path in document_paths:
    result = converter.convert(document_path)
    document = result.document
    chunks = [chunk.dict() for chunk in list(HierarchicalChunker().chunk(document))]
    final_data_list += [{'document_title': get_title(chunk['meta']['origin']['filename']), 'headings': ', '.join(chunk['meta']['headings']) if chunk['meta']['headings'] else '', 'text_content': chunk['text'], 'document_filename': chunk['meta']['origin']['filename'], 'page_number': chunk['meta']['doc_items'][0]['prov'][0]['page_no'], 'is_table': True if chunk['meta']['doc_items'][0]['label'].value.lower() == 'table' else False} for chunk in chunks if contains_valid_word_like_sequence(chunk['text'])]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]


KeyboardInterrupt



In [10]:
[chunk['meta']['headings'] for chunk in chunks]

[None,
 None,
 None,
 None,
 ['Intensivmedizin nach Polytrauma S3-Leitlinie der'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['und'],
 ['Deutsche Gesellschaft für Unfallchirurgie (DGU)'],
 ['Deutsche Gesellschaft für Unfallchirurgie (DGU)'],
 ['Deutsche Gesellschaft für Unfallchirurgie (DGU)'],
 ['Deutsche Gesellschaft für Unfallchirurgie (DGU)'],
 ['Deutsche Gesellschaft für Unfallchirurgie (DGU)'],
 ['Deutsche Gesellschaft für Unfallchirurgie (DGU)'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Herausgebende'],
 ['Bitte wie folgt zitieren:'],
 ['Bitte wie folgt zitieren:'],
 ['Die wichtigsten Empfehlungen auf einen Blick'],
 ['Die wichtigsten Empfehlungen auf einen Blick'],
 ['Die wichtigsten Empfehlungen auf einen Blick'],
 ['Die wichtigsten Empfehl

In [11]:
final_data_list

[{'document_title': 'S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
  'headings': 'Intensivmedizin (DAS -Leitlinie 2020)',
  'text_content': 'AWMF-Registernummer: 001/012',
  'document_filename': '001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf',
  'page_number': 1,
  'is_table': False},
 {'document_title': 'S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
  'headings': 'Federführende Fachgesellschaften',
  'text_content': 'Deutsche Gesellschaft für Anästhesiologie und Intensivmedizin (DGAI) Deutsche Interdisziplinäre Vereinigung für Intensiv- und Notfallmedizin (DIVI)',
  'document_filename': '001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf',
  'page_number': 1,
  'is_table': False},
 {'document_title': 'S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
  'headings': 'Beteiligte Fachgesellschaften:',
  'text_content': 'Deutsche Gesellschaft für Fachkr

In [12]:
for row in final_data_list:
    row['section_title'] = get_section_titles(row['page_number'],ExtractedContent.contents[row['document_filename'].lower()])
torch.set_default_device('cuda')

In [13]:
final_data_list

[{'document_title': 'S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
  'headings': 'Intensivmedizin (DAS -Leitlinie 2020)',
  'text_content': 'AWMF-Registernummer: 001/012',
  'document_filename': '001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf',
  'page_number': 1,
  'is_table': False,
  'section_title': ''},
 {'document_title': 'S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
  'headings': 'Federführende Fachgesellschaften',
  'text_content': 'Deutsche Gesellschaft für Anästhesiologie und Intensivmedizin (DGAI) Deutsche Interdisziplinäre Vereinigung für Intensiv- und Notfallmedizin (DIVI)',
  'document_filename': '001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf',
  'page_number': 1,
  'is_table': False,
  'section_title': ''},
 {'document_title': 'S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
  'headings': 'Beteiligte Fachgesellschaften:',
  'te

In [14]:

dataset = Dataset.from_list(final_data_list)

In [15]:
dataset

Dataset({
    features: ['document_title', 'headings', 'text_content', 'document_filename', 'page_number', 'is_table', 'section_title'],
    num_rows: 2249
})

In [16]:
dataset.save_to_disk('data/preprocessed_rag_chunks_no_embedding')

Saving the dataset (0/1 shards):   0%|          | 0/2249 [00:00<?, ? examples/s]

In [12]:
dataset = Dataset.load_from_disk('data/preprocessed_rag_chunks_no_embedding')

In [13]:
dataset.features

{'document_title': Value(dtype='string', id=None),
 'headings': Value(dtype='string', id=None),
 'text_content': Value(dtype='string', id=None),
 'document_filename': Value(dtype='string', id=None),
 'page_number': Value(dtype='int64', id=None),
 'is_table': Value(dtype='bool', id=None),
 'section_title': Value(dtype='string', id=None)}

In [15]:
dataset = dataset.add_column('embedded_text',['\n\n'.join([f"**{chunk['headings']}**",f"{chunk['text_content']}"]) for chunk in dataset])

In [16]:
def calculate_embedding(batch) -> np.ndarray: 
    batch['embedding'] = embedding_model.encode(batch['embedded_text'], task=CFG.task)
    return batch

In [17]:
tokens = embedding_tokenizer(dataset[0]['embedded_text'], return_tensors='pt', truncation=True)
embedding_model(**tokens).pooler_output.squeeze().cpu().float().numpy().shape

(1024,)

In [18]:
dataset = dataset.map(calculate_embedding, batched=True, batch_size=32)

Parameter 'function'=<function calculate_embedding at 0x000001B883A68A40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2249 [00:00<?, ? examples/s]

In [19]:
len(dataset[0]['embedding'])

1024

In [20]:
dataset.save_to_disk('data/preprocessed_rag_chunks_text_and_heading')

Saving the dataset (0/1 shards):   0%|          | 0/2249 [00:00<?, ? examples/s]